In [21]:
#https://git.aps.anl.gov/rodolakis/Macros_29id/-/blob/master/hklpy/Test_Bluesky_20210616.ipynb
# cd /home/beams/29IDUSER/Documents/User_Macros/
# %run Macros_29id/ScanFunctions_IEX.py

import databroker
from apstools.utils import listruns
from IEX_29id.utils.initialize import *
from IEX_29id.devices.detectors import *
from IEX_29id.devices.beamline_energy import *
from IEX_29id.devices.kappa_motors import *
from IEX_29id.devices.kappa_motors import kappa_motors
from IEX_29id.devices.detectors import scaler
from IEX_29id.devices.slits import *
from bluesky.plans import scan
from bluesky.callbacks import LiveTable
from bluesky.callbacks.fitting import PeakStats
from bluesky.callbacks.mpl_plotting import plot_peak_stats
import matplotlib.pyplot as plt
import numpy as np
import numpy.polynomial.polynomial as poly

print(list(databroker.catalog))
cat=databroker.catalog['29idd'];print('Number of scans in catalog: '+str(len(cat)))



x_motor=kappa_motors.m2
y_motor=kappa_motors.m3
z_motor=kappa_motors.m4
kphi_motor=kappa_motors.m1
kap_motor=kappa_motors.m7
kth_motor=kappa_motors.m8
tth_motor=kappa_motors.m9
srs1=srs.A1
srs2=srs.A2
srs3=srs.A3
srs4=srs.A4

komega=57.322396707958816   #sample 1
komega=57.82183420427256    #sample 2

scaler.stage_sigs['preset_time']=0.1

sd.baseline.append(undulator)
sd.baseline.append(mono)
sd.baseline.append(centroid_m3r)
sd.baseline.append(x_motor)
sd.baseline.append(y_motor)
sd.baseline.append(z_motor)
sd.baseline.append(tth_motor)
sd.baseline.append(kth_motor)
sd.baseline.append(kap_motor)
sd.baseline.append(kphi_motor)
sd.baseline.append(select_detector)
sd.baseline.append(srs4)




I Sun-13:35:29 - ############################################################ startup
I Sun-13:35:29 - logging started
I Sun-13:35:29 - logging level = 10


['29idd', '29idd_retired_2021_12_01']
Number of scans in catalog: 155


In [29]:
from os.path import join, isfile, exists, dirname
import ast

from ophyd import EpicsSignal
grating=EpicsSignal('29idmonoGRT_TYPE_MON', name='grating', string=True)
density=EpicsSignal('29idmono:GRT_DENSITY', name='grating')






def ID_Coef(grt,mode,hv):    # Mode = state (0=RCP,1=LCP,2=V,3=H); 
    
    """Return the ID coeff for a given polarization mode and energy;
    with Mode = 0 (RCP),1 (LCP), 2 (V), 3 (H).
    Current coefficient dictionary:
        /home/beams22/29IDUSER/Documents/User_Macros/Macros_29id/IEX_Dictionaries/Dict_IDCal.txt
    now:/home/beams3/RODOLAKIS/src/macros_29id/IEX_29id/dict/
   """
    print(grt,mode,hv)
    
    if isinstance(grt,int): grt  = {1200:'MEG',2400:'HEG'}[grt]
    if isinstance(mode,str):mode = {'RCP':0,'LCP':1,'V':2,'H':3}[mode]
    
    def read_dict(FileName,FilePath="/home/beams3/RODOLAKIS/src/macros_29id/IEX_29id/dict/"):
        with open(join(FilePath, FileName)) as f:
            for c,line in enumerate(f.readlines()):
                if line[0] == '=':
                    lastdate=line[8:16]
                lastline=line
            mydict=ast.literal_eval(lastline)
        return mydict

    
    def ListRange(grt,mode,IDdict):  # extract the list of break pts for a given mode/grt 
        tmp_list=[]
        for item in (IDdict[grt][mode]):
            tmp_list.append(item[0])  
        return tmp_list


    def FindRange(hv,range_list):         # returns the index for the corresponding range
        B = [x - hv for x in range_list]
        index = [i for (i, x) in enumerate(B) if x > 0]
        return(index[0])
    
    try:
        #FRPath = '/Users/fannysimoes/Box/6-Python/MyPython/Macros_29id/'   #   FR hardcoded
        ID_function=read_dict(FileName='Dict_IDCal.txt')
    
    except KeyError:
        print("Unable to read dictionary") 
        
    try:   
        Lrange = ListRange(grt,mode,ID_function)
        Erange = FindRange(hv,Lrange)
        K = ID_function[grt][mode][Erange][1]
        
    except KeyError:
        print("WARNING: PLease select one of the following:")
        print("        mode 0 = RCP")
        print("        mode 1 = LCP")
        print("        mode 2 = V")
        print("        mode 3 = H")
        
    return K



def ID_Calc(grt,mode,hv):    # Mode = state (0=RCP,1=LCP,2=V,3=H)
    """Calculate the ID SP for a given polarization mode and energy;
    with Mode = 0 (RCP),1 (LCP), 2 (V), 3 (H)"""
    if isinstance(grt,int): grt  = {1200:'MEG',2400:'HEG'}[grt]
    if isinstance(mode,str):mode = {'RCP':0,'LCP':1,'V':2,'H':3}[mode]
    try:
        K=ID_Coef(grt,mode,hv)
        #ID = K[0] + K[1]*hv**1 + K[2]*hv**2 + K[3]*hv**3 + K[4]*hv**4 + K[5]*hv**5
        ID=poly.polyval(hv,K)
    except KeyError:
        print("WARNING: PLease select one of the following:")
        print("        mode 0 = RCP")
        print("        mode 1 = LCP")
        print("        mode 2 = V")
        print("        mode 3 = H")
        ID=caget("ID29:EnergySeteV")
    return round(ID,1)


def SetRange(hv):
    Mode=caget("ID29:ActualMode")
    hv_min=ID_Range()[1]
    hv_max=ID_Range()[2]
    sleep(0.1)
    hv_SP=min(round(max(hv,hv_min)*1.0,3),hv_max)*1.0
    sleep(0.1)
#    if round(hv,1) != hv_SP:
    if hv < hv_min or hv > hv_max:
        print("\nWARNING: Set point out of BL energy range:")
    return hv_SP



# def ID_Range():      # mode = state (0=RCP,1=LCP,2=V,3=H)
#     #Mode=caget("ID29:ActualMode")
#     mode=undulator.actual_mode.get()
#     #GRT=caget("29idmono:GRT_DENSITY")
#     grt=density.get()
#     ID_min=[400,400,440,250,250]
#     ID_max=3800
#     hv_min=[380,380,420,245,240]
#     if grt == 2400:
#         hv_max=2000
#     elif grt == 1200:
#         hv_max=3000
#     return ID_min[mode],hv_min[mode],hv_max,ID_max


# def Energy_Range(grating_density,undulator_mode):
#     BL_range={}    #   Slit:PE
#     BL_range[2400]  = { "H":(250,2000), "V":(440,2000), "RCP":(400,2000), "LCP":(400,2000)  }
#     BL_range[1200]  = { "H":(250,2500), "V":(440,2500), "RCP":(400,2500), "LCP":(400,2500) }
#     energy_min, energy_max = BL_range[grating_density][undulator_mode]
#     return energy_min, energy_max






In [5]:
read_dict('Dict_Slit.txt')

{'HEG': {'S1H': -0.125, 'S1V': 0.0, 'S2H': -0.25, 'S2V': -0.25},
 'MEG': {'S1H': 0.125, 'S1V': 0.0, 'S2H': -0.25, 'S2V': 0.0}}

In [6]:
from ophyd import EpicsSignal
grating=EpicsSignal('29idmonoGRT_TYPE_MON', name='grating', string=True)
density=EpicsSignal('29idmono:GRT_DENSITY', name='grating')

In [7]:
density.get()

1200.0

In [8]:
undulator.actual_mode.get()

2

In [44]:
mono.grating_density.value

1200.0

In [15]:
if isinstance(Out[11],str): 
    print('yes')

yes


In [16]:
if isinstance(Out[14],str): 
    print('yes')

In [39]:
mode=undulator.actual_mode_str.get()

In [40]:
mode

'V'

In [44]:
mode=undulator.actual_mode.get()

In [42]:
mode

2

In [45]:
RE(bps.mv(mono.energy,685))

()

In [19]:
ID_Range()

(440, 430, 3000, 3800)

In [21]:
ID_Coef('MEG',2,536)

[-604.5006436697821,
 3.0530278050672597,
 0.0006285185258970196,
 -7.876285868395855e-06,
 6.618688516815772e-09]

In [27]:
ID_Calc(1200,2,680)

1200 2 680


692.7

In [27]:
Slit_Coef(1)

('29id:k_slit1A', (2.3325, -0.000936, 2.4e-07, 2.3935, -0.0013442, 3.18e-07))

In [28]:
Slit_Coef(2)

('29id:k_slit2B', (3.61, -0.00186, 5.2e-07, 6.8075, -0.003929, 9.5e-07))

In [38]:
mono.energy

MonoEnergy(prefix='29idmono:', name='mono_energy', parent='mono', settle_time=0.0, timeout=None, read_attrs=['setpoint', 'readback'], configuration_attrs=[], limits=None, egu='')

In [38]:
density.value

1200.0

In [42]:
def SetSlit_BL(c2B=1,c1A=1,q=None):

    RBV = mono.energy.readback.value
    GRT= density.value
    
    hv=max(RBV,500)
    hv=min(RBV,2000)
    c=4.2/2.2

    if GRT==1200:
        GRT='MEG'
        V=0.65        #  set to 65% of RR calculation for both grt => cf 2016_2_summary
    elif GRT==2400:
        GRT='HEG'
        V=0.65*c        #  set to 65% of RR calculation (no longer 80%) => cf 2016_2_summary

    try:
        slit_position=read_dict(FileName='Dict_Slit.txt')
    except KeyError:
        print("Unable to read dictionary")
        return

    V2center= slit_position[GRT]['S2V']
    H2center= slit_position[GRT]['S2H']
    V1center= slit_position[GRT]['S1V']
    H1center= slit_position[GRT]['S1H']

    Size1A=( Aperture_Fit(hv,1)[0]*c1A,       Aperture_Fit(hv,1)[1]*c1A )
    Size2B=( Aperture_Fit(hv,2)[0]*c2B, round(Aperture_Fit(hv,2)[1]*c2B*V,3))
    #SetSlit1A(Size1A[0],Size1A[1],H1center,V1center,q)    # standard operating
    #SetSlit2B(Size2B[0],Size2B[1],H2center,V2center,q)
    return Size1A, Size2B


def Aperture_Fit(hv,n):
    K=Slit_Coef(n)[1]
    sizeH=K[0]+K[1]*hv+K[2]*hv*hv
    sizeV=K[3]+K[4]*hv+K[5]*hv*hv
    return [round(sizeH,3),round(sizeV,3)]


In [43]:
SetSlit_BL(1)

((1.901, 1.767), (2.766, 3.238))

In [2]:
slits_calc(0.75)

(1.901, 1.767, 0.125, 0.0, 2.0745, 2.429, -0.25, 0.0)

In [4]:
RE(slitBL_set(1))

()

In [8]:
GRT=1200
c=4.2/2.2

In [9]:
GRT,V={1200:('MEG',0.65),2400:('HEG',0.65*c)}[GRT]

In [10]:
GRT

'MEG'

In [11]:
V

0.65

In [ ]:
v={0:1,1:2,2:5,3:10,4:20,5:50,6:100,7:200,8:500}
u={0:'pA',1:'nA',2:'uA',4:'mA'}

In [17]:

def d4_gain_set(gain_value,gain_unit,n=4):
    v={1:0,2:1,5:2,10:3,20:4,50:5,100:6,200:7,500:8}[gain_value]
    u={'pA':0,'nA':1,'uA':2,'mA':4}[gain_unit]
    if n==1:
        srs1.sensitivity_unit.put(u); RE(bps.sleep(1));u=srs1.sensitivity_unit.get()
        srs1.sensitivity_value.put(v);RE(bps.sleep(1));v=srs1.sensitivity_value.get()
    elif n==2:
        srs2.sensitivity_unit.put(u); RE(bps.sleep(1));u=srs2.sensitivity_unit.get()
        srs2.sensitivity_value.put(v);RE(bps.sleep(1));v=srs2.sensitivity_value.get()
    elif n==3:
        srs3.sensitivity_unit.put(u); RE(bps.sleep(1));u=srs3.sensitivity_unit.get()
        srs3.sensitivity_value.put(v);RE(bps.sleep(1));v=srs3.sensitivity_value.get()
    elif n==4:
        srs4.sensitivity_unit.put(u); RE(bps.sleep(1));u=srs4.sensitivity_unit.get()
        srs4.sensitivity_value.put(v);RE(bps.sleep(1));v=srs4.sensitivity_value.get()
    print('SRS',n,':',v,u)

In [19]:
ID_Coef(1200,0,520)

1200 0 520


[-5940.176848239725,
 48.30636368582396,
 -0.14075467796987542,
 0.00018579017325356665,
 -9.17653113254405e-08]

In [20]:
ID_Calc(1200,0,520)

1200 0 520


NameError: name 'poly' is not defined

In [36]:




def id_coef(grt,mode,hv):    # Mode = state (0=RCP,1=LCP,2=V,3=H); 
    
    """Return the ID coeff for a given polarization mode and energy;
    with Mode = 0 (RCP),1 (LCP), 2 (V), 3 (H).
    Current coefficient dictionary:
        /home/beams22/29IDUSER/Documents/User_Macros/Macros_29id/IEX_Dictionaries/Dict_IDCal.txt
    now:/home/beams3/RODOLAKIS/src/macros_29id/IEX_29id/dict/
    """
    
    if isinstance(grt,int): grt  = {1200:'MEG',2400:'HEG'}[grt]
    if isinstance(mode,str):mode = {'RCP':0,'LCP':1,'V':2,'H':3}[mode]
    
    def read_dict(FileName,FilePath="/home/beams3/RODOLAKIS/src/macros_29id/IEX_29id/dict/"):
        with open(join(FilePath, FileName)) as f:
            for c,line in enumerate(f.readlines()):
                if line[0] == '=':
                    lastdate=line[8:16]
                lastline=line
            mydict=ast.literal_eval(lastline)
        return mydict

    
    def list_range(grt,mode,IDdict):  # extract the list of break pts for a given mode/grt 
        tmp_list=[]
        for item in (IDdict[grt][mode]):
            tmp_list.append(item[0])  
        return tmp_list


    def find_range(hv,range_list):         # returns the index for the corresponding range
        B = [x - hv for x in range_list]
        index = [i for (i, x) in enumerate(B) if x > 0]
        return(index[0])
    
    try:
        #FRPath = '/Users/fannysimoes/Box/6-Python/MyPython/Macros_29id/'   #   FR hardcoded
        ID_function=read_dict(FileName='Dict_IDCal.txt')
    
    except KeyError:
        print("Unable to read dictionary") 
        
    try:   
        Lrange = list_range(grt,mode,ID_function)
        Erange = find_range(hv,Lrange)
        K = ID_function[grt][mode][Erange][1]
        
    except KeyError:
        print("WARNING: PLease select one of the following:")
        print("        mode 0 = RCP")
        print("        mode 1 = LCP")
        print("        mode 2 = V")
        print("        mode 3 = H")
        
    return K




def id_calc(grt,mode,hv):    # Mode = state (0=RCP,1=LCP,2=V,3=H)
    """Calculate the ID SP for a given polarization mode and energy;
    with Mode = 0 (RCP),1 (LCP), 2 (V), 3 (H)"""
    if isinstance(grt,int): grt  = {1200:'MEG',2400:'HEG'}[grt]
    if isinstance(mode,str):mode = {'RCP':0,'LCP':1,'V':2,'H':3}[mode]
    try:
        K=id_coef(grt,mode,hv)
        #ID = K[0] + K[1]*hv**1 + K[2]*hv**2 + K[3]*hv**3 + K[4]*hv**4 + K[5]*hv**5
        ID=poly.polyval(hv,K)
    except KeyError:
        ID=0
        print('KeyError')
    return round(ID/1000,4)



In [37]:
id_calc(1200,3,680)

0.6968

In [49]:
align_m3r.put(1)

In [51]:
RE(bps.mv(align_m3r,1))

()

In [52]:


def id_coef(grt,mode,hv):    # Mode = state (0=RCP,1=LCP,2=V,3=H); 
    
    """Return the ID coeff for a given polarization mode and energy;
    with Mode = 0 (RCP),1 (LCP), 2 (V), 3 (H).
    Current coefficient dictionary:
        /home/beams22/29IDUSER/Documents/User_Macros/Macros_29id/IEX_Dictionaries/Dict_IDCal.txt
    now:/home/beams3/RODOLAKIS/src/macros_29id/IEX_29id/dict/
   """
    print(grt,mode,hv)
    
    if isinstance(grt,int): grt  = {1200:'MEG',2400:'HEG'}[grt]
    if isinstance(mode,str):mode = {'RCP':0,'LCP':1,'V':2,'H':3}[mode]
    
    def read_dict(FileName,FilePath="/home/beams3/RODOLAKIS/src/macros_29id/IEX_29id/dict/"):
        with open(join(FilePath, FileName)) as f:
            for c,line in enumerate(f.readlines()):
                if line[0] == '=':
                    lastdate=line[8:16]
                lastline=line
            mydict=ast.literal_eval(lastline)
        return mydict

    
    def list_range(grt,mode,IDdict):  # extract the list of break pts for a given mode/grt 
        tmp_list=[]
        for item in (IDdict[grt][mode]):
            tmp_list.append(item[0])  
        return tmp_list


    def find_range(hv,range_list):         # returns the index for the corresponding range
        B = [x - hv for x in range_list]
        index = [i for (i, x) in enumerate(B) if x > 0]
        return(index[0])
    
    try:
        #FRPath = '/Users/fannysimoes/Box/6-Python/MyPython/Macros_29id/'   #   FR hardcoded
        ID_function=read_dict(FileName='Dict_IDCal.txt')
    
    except KeyError:
        print("Unable to read dictionary") 
        
    try:   
        Lrange = list_range(grt,mode,ID_function)
        Erange = find_range(hv,Lrange)
        K = ID_function[grt][mode][Erange][1]
        
    except KeyError:
        print("WARNING: PLease select one of the following:")
        print("        mode 0 = RCP")
        print("        mode 1 = LCP")
        print("        mode 2 = V")
        print("        mode 3 = H")
        
    return K



In [53]:
id_coef('MEG',0,555)

MEG 0 555


[-5940.176848239725,
 48.30636368582396,
 -0.14075467796987542,
 0.00018579017325356665,
 -9.17653113254405e-08]